<a href="https://colab.research.google.com/github/Adefolarin-o/garbage-classification-efficientnet/blob/main/CodeFestvFinalSubmission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Garbage Classification with EfficientNetV2B0(Group 5 Codefest)

## Overview
#This project classifies images of garbage into 10 categories using transfer learning with EfficientNetV2B0.

In [ ]:
# Cell 1:
# Install required packages for the project
# - kagglehub: Download datasets from Kaggle
# - tensorflow: Core ML framework
# - keras_cv: Computer vision utilities
# ------------------------------------------
!pip install kagglehub -q
!pip install tensorflow keras keras_cv -q

import os
import shutil
import random
import numpy as np
import tensorflow as tf
from pathlib import Path
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Enable mixed precision for faster training
tf.keras.mixed_precision.set_global_policy('mixed_float16')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 650.7/650.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 30.4 MB/s eta 0:00:00


In [ ]:
# Cell 2: Download the garbage classification dataset from Kaggle Hub
#
# - Dataset: "sumn2u/garbage-classification-v2"
# - Contains 10 classes of recyclable and non-recyclable waste
# -------------------------------------------------------------------
# Download dataset
import kagglehub
dataset_path = kagglehub.dataset_download("sumn2u/garbage-classification-v2")
dataset_dir = os.path.join(dataset_path, "garbage-dataset")

# Create split directories
base_split_dir = "/content/splitted_data"
train_dir = os.path.join(base_split_dir, 'train')
val_dir = os.path.join(base_split_dir, 'val')
test_dir = os.path.join(base_split_dir, 'test')

# Clean existing splits
if os.path.exists(base_split_dir):
    shutil.rmtree(base_split_dir)
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

100%|██████████| 744M/744M [00:12<00:00, 62.2MB/s]

Extracting files...


In [ ]:
# Cell 3: Data splitting function
#
# Splits dataset into train/val/test sets while maintaining class balance.
# Args:
#       source_dir (str): Path to directory containing class folders
#       train_split (float): Proportion of data for training (default: 0.7)
#       val_split (float): Proportion of data for validation (default: 0.2)
#
#   Returns:
#       None: Creates train/val/test directories and copies images
#--------------------------------------------------------------------------
def split_data(source_dir, train_split=0.7, val_split=0.2):
    classes = [d for d in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, d))]

    for cls in classes:
        # Create target directories
        os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
        os.makedirs(os.path.join(val_dir, cls), exist_ok=True)
        os.makedirs(os.path.join(test_dir, cls), exist_ok=True)

        # Get and shuffle images
        src = os.path.join(source_dir, cls)
        images = list(Path(src).glob('*.[jpJP][pnPN]*'))  # All jpg/png
        random.shuffle(images)

        # Split indices
        n = len(images)
        train_end = int(n * train_split)
        val_end = train_end + int(n * val_split)

        # Copy files (The test set is formed implicitly from the val_end to the end)
        for img in images[:train_end]:
            shutil.copy(img, os.path.join(train_dir, cls, img.name))
        for img in images[train_end:val_end]:
            shutil.copy(img, os.path.join(val_dir, cls, img.name))
        for img in images[val_end:]:
            shutil.copy(img, os.path.join(test_dir, cls, img.name))

split_data(dataset_dir)
print("Data splitting complete!")

Data splitting complete!


In [ ]:
# Cell 4: Verify class distribution
#
#   Prints the number of samples per class in a given directory.
#
#   Args:
#       split_dir (str): Path to directory containing class folders
#
#   Returns:
#       None: Prints class distribution to console
# --------------------------------------------------
def check_class_distribution(split_dir):

    print(f"\nClass distribution in {split_dir}:")
    for cls in os.listdir(split_dir):
        cls_path = os.path.join(split_dir, cls)
        if os.path.isdir(cls_path):
            print(f"{cls}: {len(os.listdir(cls_path))} samples")

check_class_distribution(train_dir)
check_class_distribution(val_dir)
check_class_distribution(test_dir)


Class distribution in /content/splitted_data/train:
shoes: 1383 samples
plastic: 1388 samples
metal: 714 samples
paper: 1176 samples
clothes: 3728 samples
biological: 697 samples
glass: 2142 samples
trash: 662 samples
cardboard: 1277 samples
battery: 660 samples

Class distribution in /content/splitted_data/val:
shoes: 395 samples
plastic: 396 samples
metal: 204 samples
paper: 336 samples
clothes: 1065 samples
biological: 199 samples
glass: 612 samples
trash: 189 samples
cardboard: 365 samples
battery: 188 samples

Class distribution in /content/splitted_data/test:
shoes: 199 samples
plastic: 200 samples
metal: 102 samples
paper: 168 samples
clothes: 534 samples
biological: 101 samples
glass: 307 samples
trash: 96 samples
cardboard: 183 samples
battery: 96 samples


In [ ]:
# Cell 5: Data preprocessing and generators
#
# - Configure data augmentation to improve model generalization
# - Random rotations (up to 30 degrees) simulate real-world variations
# - Shifts and zooms account for imperfect camera angles
# - Horizontal flips add diversity for symmetric objects
# --------------------------------------------------------------------
from tensorflow.keras.applications.efficientnet import preprocess_input

# Image parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 64  # Increased batch size for T4 memory

# Data augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # EfficientNet-specific preprocessing, e.g corrects imagepixel sizes for the model
    rotation_range=30, # Rotate images randomly within ±30 degrees
    width_shift_range=0.2, # Shift images horizontally by up to 20%
    height_shift_range=0.2, # Shift images vertically by up to 20%
    shear_range=0.2, # Shear transformations for perspective effects
    zoom_range=0.2, # Randomly zoom in/out by up to 20%
    horizontal_flip=True, # Randomly flip images horizontally
    fill_mode='nearest' # Fill missing pixels using nearest neighbor
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 13827 images belonging to 10 classes.
Found 3949 images belonging to 10 classes.
Found 1986 images belonging to 10 classes.


In [ ]:
# Cell 6: Model building
#
#   Builds a transfer learning model using EfficientNetV2B0 as the base.
#
#    Args:
#        num_classes (int): Number of output classes (default: 10)
#
#    Returns:
#        model (tf.keras.Model): Compiled model ready for training
#
    # Load EfficientNetV2B0 with pre-trained ImageNet weights
    # - include_top=False: Exclude the final classification layer
    # - input_shape: Matches image size (224x224) with 3 color channels
# --------------------------------------------------
def build_model(num_classes=10):

    # Base model with frozen weights
    base_model = EfficientNetV2B0(
        include_top=False,
        weights='imagenet',
        input_shape=(*IMG_SIZE, 3)
    )
    base_model.trainable = False

    # Custom head
    inputs = tf.keras.Input(shape=(*IMG_SIZE, 3))
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu', dtype='float32')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax', dtype='float32')(x)

    model = tf.keras.Model(inputs, outputs)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

model = build_model()
model.summary()

24274472/24274472 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ efficientnetv2-b0 (Functional)       │ (None, 7, 7, 1280)          │       5,919,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ cast_1 (Cast)                        │ (None, 1280)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │         327,936 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ cast_2 (Cast)                        │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ cast_3 (Cast)                        │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           2,570 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,249,818 (23.84 MB)

 Trainable params: 330,506 (1.26 MB)

 Non-trainable params: 5,919,312 (22.58 MB)

In [ ]:
# Cell 7: First training phase (frozen base)
# Configure early stopping to prevent overfitting
# - Monitors validation accuracy
# - Stops training if no improvement for 6 epochs
# - Restores weights from the best epoch
# --------------------------------------------------
early_stop = callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=6,
    restore_best_weights=True
)

reduce_lr = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=2,
    min_lr=1e-6
)

print("Training phase 1 (frozen base)...")
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=[early_stop, reduce_lr]
)

Training phase 1 (frozen base)...


KeyboardInterrupt: 

In [ ]:
# Cell 8: Fine-tuning phase (partial unfreeze) - CORRECTED
# - Unfreeze the top 20% of base model layers for fine-tuning
# - Allows the model to adapt low-level features to the garbage dataset
# - Keeps bottom 80% frozen to preserve generic ImageNet features
# --------------------------------------------------
# Unfreeze top 20% of base layers
model.layers[1].trainable = True
for layer in model.layers[1].layers[:int(len(model.layers[1].layers)*0.8)]:
    layer.trainable = False

# Re-compile with lower learning rate
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Reset callbacks for fresh start
early_stop_fine = callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True
)

reduce_lr_fine = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=2,
    min_lr=1e-7
)

print("\nTraining phase 2 (fine-tuning)...")
history_fine = model.fit(
    train_generator,
    epochs=15,  # Train for 15 NEW epochs
    # Remove initial_epoch parameter completely
    validation_data=val_generator,
    callbacks=[early_stop_fine, reduce_lr_fine]
)

In [ ]:

# Cell 9: Evaluation
# - Evaluate the model on the test set
# - Measures generalization performance on unseen data
# - Reports accuracy and loss for final assessment
# --------------------------------------------------
print("\nFinal evaluation:")
test_loss, test_acc = model.evaluate(test_generator)
print(f"\nTest accuracy: {test_acc:.2%}")
print(f"Test loss: {test_loss:.4}")



Final evaluation:
32/32 ━━━━━━━━━━━━━━━━━━━━ 13s 419ms/step - accuracy: 0.9727 - loss: 0.0876

Test accuracy: 97.08%
Test loss: 0.08813


In [ ]:
#Cell 10: Mounting Google Drive
# -----------------------------
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
#Cell 11: Saving the Model
# - Saves the trained model to Google Drive for future use
# - Uses the modern .keras format (recommended for Keras 3+)
# - Includes architecture, weights, and optimizer state
# ---------------------------------------------------------
model.save("/content/drive/MyDrive/garbage_classifier_v2.keras")
print("Model saved to Google Drive!")

Model saved to Google Drive!


In [ ]:
#Cell 12: Verifying that the Model is Saved
# -----------------------------------------
import os

model_path = "/content/drive/MyDrive/garbage_classifier_v2.keras"  # Update with the correct path
if os.path.exists(model_path):
    print("File found!")
else:
    print("File not found. Please check the path.")

File found!


In [ ]:
#Cell 13: Importing and Loading the Model
#----------------------------------------
from tensorflow import keras

model_path = "/content/drive/MyDrive/garbage_classifier_v2.keras"

# Load the model
loaded_model = keras.models.load_model("/content/drive/MyDrive/garbage_classifier_v2.keras")
print("Model loaded successfully!")


Model loaded successfully!


In [ ]:
# Cell 14: Interactive Test Explorer
# - Create interactive widgets for exploring test predictions
# - Dropdown: Select a test sample by index
# - Slider: Adjust confidence threshold for predictions
# --------------------------------------------------
from IPython.display import display, HTML
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np

# Get test image paths and labels
test_files = test_generator.filepaths
class_names = list(test_generator.class_indices.keys())
true_labels = test_generator.labels

# Create widgets
test_sample_selector = widgets.Dropdown(
    options=[(f"Sample {i}", i) for i in range(len(test_files))],
    description='Select Sample:',
    style={'description_width': 'initial'}
)

confidence_threshold = widgets.FloatSlider(
    value=0.7,
    min=0,
    max=1.0,
    step=0.05,
    description='Confidence Threshold:',
    style={'description_width': 'initial'}
)

prediction_output = widgets.Output()

def update_test_sample(change):

#   Updates the displayed test sample and prediction results.
#
#   Args:
#       change: Widget change event (automatically passed by IPython)

    with prediction_output:
        prediction_output.clear_output(wait=True)

        idx = test_sample_selector.value
        img_path = test_files[idx]
        true_class = class_names[true_labels[idx]]

        # Load and preprocess image
        img = tf.keras.utils.load_img(img_path, target_size=IMG_SIZE)
        img_array = tf.keras.utils.img_to_array(img)
        img_array = preprocess_input(img_array[np.newaxis, ...])

        # Make prediction
        probs = loaded_model.predict(img_array, verbose=0)[0]
        pred_class = class_names[np.argmax(probs)]
        confidence = np.max(probs)

        # Display the selected image with true and predicted labels
        plt.figure(figsize=(8, 6))
        plt.imshow(img)
        plt.title(f"True: {true_class}\nPredicted: {pred_class} ({confidence:.1%})")
        plt.axis('off')
        plt.show()

        # Plot the model's confidence distribution across all classes
        plt.figure(figsize=(10, 3))  # Wide figure for better readability
        sorted_indices = np.argsort(probs)[::-1]  # Sort classes by confidence
        colors = ['green' if (p >= confidence_threshold.value) else 'gray' for p in probs[sorted_indices]]  # Color coding
        plt.bar(range(len(probs)), probs[sorted_indices], color=colors)  # Bar plot
        plt.xticks(range(len(probs)), [class_names[i] for i in sorted_indices], rotation=45)  # Class labels
        plt.ylabel("Confidence")  # Y-axis label
        plt.axhline(confidence_threshold.value, color='red', linestyle='--')  # Threshold line
        plt.show()  # Render the plot


# Connect widgets to function
test_sample_selector.observe(update_test_sample, names='value')
confidence_threshold.observe(update_test_sample, names='value')

# Display interface
print("\n🔍 Interactive Test Explorer")
display(widgets.VBox([test_sample_selector, confidence_threshold]))
display(prediction_output)

# Trigger initial update
update_test_sample(None)


🔍 Interactive Test Explorer


Output()